# C3S examples 
## Retrieve and visualize CSV and Shape files

## 1 - Configuration

### 1.1 - SET YOUR C3S ACCOUNT (FOR THE AUTHENTIFICATION) 

In [ ]:
# For Registration (C3S)
url_registration= 'https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome'
from IPython.core.display import display, HTML
display(HTML(f"""<a href={url_registration}>{'Registration_C3S - please click on it'}</a>"""))

### 1.2 - Log in and retrieve your key

In [ ]:
url_login= 'https://cds.climate.copernicus.eu/user/login?destination=%2F%23!%2Fhome'
url_key = 'https://cds.climate.copernicus.eu/api-how-to'

display(HTML(f"""<a href={url_login}>{'Please login - please click on it'}</a>"""))
display(HTML(f"""<a href={url_key}>{'Retrieve your key - please click on it'}</a>"""))

### 1.3- Install the CDS API key

In [ ]:
import os

print('Please enter your key:')
key = input()

content = "url: https://cds.climate.copernicus.eu/api/v2\nkey: " + key

#Write content in $HOME/.cdsapirc 
f = open(os.environ["HOME"] + '/cdsapirc', 'w')
f.write(content)
f.close()
!mv $HOME/cdsapirc $HOME/.cdsapirc

## 2 - Consult the catalogue

#### Choose a dataset, click on "Download data", select the desired parameters, agree to the Terms of Use and finally, click on "Show API request" to see the Python script

In [ ]:
url_dataset= 'https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset'
display(HTML(f"""<a href={url_dataset}>{'Consult the catalogue - please click on it'}</a>"""))

## 3 - Satellite Observations

### 3.1 - Create a data folder

In [ ]:
#Get the destination folder path
folder = '/home/jovyan/c3s_data/c3s_csv_shapefiles_examples'
import os
#Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

zip_file = folder + '/glaciers_mass_balance' #Give a name to your file

### 3.2 - Download a file
#### TO DO : Check you agree with the __[Terms of Use](https://cds.climate.copernicus.eu/cdsapp#!/dataset/insitu-glaciers-elevation-mass?tab=form)__ (Last section)
#### Note : Please, go to https://cds.climate.copernicus.eu/cdsapp#!/yourrequests to track the progress of your request

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'insitu-glaciers-elevation-mass',
    {
        'variable':'all',
        'format':'zip',
        'product_type':'mass_balance',
        'file_version':'20181103'
    },
    zip_file)

### 3.3 - Unzip the file

In [ ]:
import zipfile

zip = zipfile.ZipFile(zip_file)
zip.extractall(folder)

### 3.4 - Display a map

In [ ]:
from folium.plugins import MarkerCluster
import pandas
import folium
import branca
import shapefile
import numpy as np
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import datetime
from io import BytesIO
import base64

#Read the files
csv = pandas.read_csv(folder + '/_C3S_MASS_BALANCE_DATA_20181103.csv', encoding='ISO-8859-1')
sf = shapefile.Reader( folder + '/_C3S_MASS_BALANCE_SERIES_20181103.shp')

#Create a progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=len(sf.records()),
    step=1,
    description='Loading:',
)
display(progress_bar)

#Create a map
m = folium.Map(location=[20,0], zoom_start=5)
marker_cluster = MarkerCluster().add_to(m)

#For each record, add a marker to the map
for feature in sf.records():
    id = feature.WGMS_ID
    
    #Retrieve information in the csv file
    survey_dates = [datetime.strptime(str(i)[0:4], '%Y') \
                    for i in list(csv.SURVEY_DATE[csv.WGMS_ID == id])]
    mass_balances = list(csv.ANNUAL_BALANCE[csv.WGMS_ID == id])

    #Create a figure
    f= figure()
    
    plt.bar(range(len(mass_balances)), mass_balances, alpha=0.5)
    plt.xticks(np.arange(0, len(mass_balances), step=len(survey_dates)//10+1), \
               [rd.year for rd in survey_dates[::len(survey_dates)//10+1]])
    
    plt.title(csv.NAME[csv.WGMS_ID == id].any() + ' WGMS_ID=' + str(id))
    plt.xlabel('Year')
    plt.ylabel('Annual mass balance')
    
    buf = BytesIO()
    f.savefig(buf, format="png")
    plt.close(f)
    
    #Embed the result in the html output
    data = base64.b64encode(buf.getbuffer()).decode("ascii")
    html = f"<img src='data:image/png;base64,{data}' style='max-width: 100%;max-height: 100%;'/>" 
    
    #Create a popup
    iframe = branca.element.IFrame(html=html, width=500, height=300)
    popup = folium.Popup(iframe, max_width=800)

    #Create a marker 
    folium.Marker(location=[feature.LATITUDE, feature.LONGITUDE],\
                  popup=popup).add_to(marker_cluster)
    
    progress_bar.value += 1

#Save the map
m.save(folder + '/folium_map.html')

#### TO DO : Please Copy The Download Link Of The File "folium_map.html" And Paste It In Url

In [ ]:
Url=''
info='Please Click on it to display the result'
from IPython.core.display import display, HTML

display(HTML(f"""<a href={Url}>{info}</a>"""))